In [17]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [18]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-04-17 22:45:24,-0.6000,73.0833,82.53,76,54,6.71
1,1,Gornozavodsk,RU,2022-04-17 22:45:24,58.3758,58.3211,19.24,95,10,2.84
2,2,Rawson,AR,2022-04-17 22:45:24,-43.3002,-65.1023,60.67,74,1,24.49
3,3,Mahebourg,MU,2022-04-17 22:45:25,-20.4081,57.7000,77.29,94,75,0.00
4,4,Georgetown,MY,2022-04-17 22:45:10,5.4112,100.3354,82.33,99,40,2.30


In [19]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [20]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [21]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure()
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-04-17 22:45:24,-0.6000,73.0833,82.53,76,54,6.71
3,3,Mahebourg,MU,2022-04-17 22:45:25,-20.4081,57.7000,77.29,94,75,0.00
4,4,Georgetown,MY,2022-04-17 22:45:10,5.4112,100.3354,82.33,99,40,2.30
7,7,Vanimo,PG,2022-04-17 22:45:26,-2.6741,141.3028,80.67,86,88,2.95
9,9,Vaini,TO,2022-04-17 22:41:46,-21.2000,-175.2000,84.36,79,75,6.91
11,11,Bikaner,IN,2022-04-17 22:45:27,28.0167,73.3000,84.40,21,34,10.42
13,13,Meulaboh,ID,2022-04-17 22:45:27,4.1363,96.1285,77.25,89,98,3.02
16,16,Isangel,VU,2022-04-17 22:45:28,-19.5500,169.2667,82.06,74,100,1.61
17,17,Butaritari,KI,2022-04-17 22:45:28,3.0707,172.7902,81.77,74,12,11.81
18,18,Bubaque,GW,2022-04-17 22:45:29,11.2833,-15.8333,75.20,76,38,20.51


In [28]:
preferred_cities_df.count()

City_ID       160
City          160
Country       160
Date          160
Lat           160
Lng           160
Max Temp      160
Humidity      160
Cloudiness    160
Wind Speed    160
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.53,-0.6000,73.0833,
3,Mahebourg,MU,77.29,-20.4081,57.7000,
4,Georgetown,MY,82.33,5.4112,100.3354,
7,Vanimo,PG,80.67,-2.6741,141.3028,
9,Vaini,TO,84.36,-21.2000,-175.2000,
11,Bikaner,IN,84.40,28.0167,73.3000,
13,Meulaboh,ID,77.25,4.1363,96.1285,
16,Isangel,VU,82.06,-19.5500,169.2667,
17,Butaritari,KI,81.77,3.0707,172.7902,
18,Bubaque,GW,75.20,11.2833,-15.8333,


In [43]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
    
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.53,-0.6000,73.0833,Scoop Guest House
3,Mahebourg,MU,77.29,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Georgetown,MY,82.33,5.4112,100.3354,Cititel Penang
7,Vanimo,PG,80.67,-2.6741,141.3028,Vahmoneh Lodge Vanimo
9,Vaini,TO,84.36,-21.2000,-175.2000,Keleti Beach Resort
11,Bikaner,IN,84.40,28.0167,73.3000,The Laxmi Niwas Palace
13,Meulaboh,ID,77.25,4.1363,96.1285,Kost Pria
16,Isangel,VU,82.06,-19.5500,169.2667,Tanna Lodge
17,Butaritari,KI,81.77,3.0707,172.7902,Isles Sunset Lodge
18,Bubaque,GW,75.20,11.2833,-15.8333,"Casa Dora, Bubaque"


In [47]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [50]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))